In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.optim import Optimizer
#from adam import Adam

In [30]:
#функция-обучения нейронной сети
def train_loop(data, model, func_loss, optimizer):
    size = len(data.dataset)

    model.train()
    running_loss = 0.0
    for batch, (inputs, labels) in enumerate(data, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = func_loss(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (batch + 1) % 2000 == 0:
            print(f"loss: {running_loss / 2000:>7f}  [{batch + 1:>5d}/{size:>5d}]")
            running_loss = 0.0


In [31]:
#функция-тестирование нейронной сети
def test_loop(data, model, func_loss):
    correct = 0
    total = 0
    test_loss = 0

    model.eval()
    with torch.no_grad():
        for (inputs, labels) in data:
            outputs = model(inputs)
            test_loss += func_loss(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%, loss: %.7f' % (100 * correct / total, test_loss / (total / batch_size)))

In [32]:
#выбор гиперпараметров
batch_size = 4
learn_rate = 1e-3
epoch = 10
momentum = 0.9

In [33]:
#Преобразование данных: перевод в тензор из PyTorch и нормализация
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#загрузка датасета
data_train = datasets.CIFAR10(root = "./data", train = True, download = True, transform = transform)
data_test = datasets.CIFAR10(root = "./data", train = False, download = False, transform = transform)
#разбитие данных на партии
dataload_train = DataLoader(data_train, batch_size = batch_size, shuffle = True)
dataload_test = DataLoader(data_test, batch_size = batch_size, shuffle = True)

Files already downloaded and verified


In [34]:
#представление модели нейронной сети
class NeuralNetworks(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120,),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, input):
        output = self.network(input)
        return output

In [101]:
class Adam(Optimizer):
    def __init__(self, param, alpha = 1e-3, beta1 = 0.9, beta2 = 0.999, eps = 1e-8):
        self.param = list(param)
        number_of_param = sum([par.numel() for par in self.param])
        # print(number_of_param)
        self.t = 0
        self.m = [0] * number_of_param
        self.v = [0] * number_of_param
        defaults = dict(lr=alpha, betas=(beta1, beta2), eps=eps)
        super().__init__(self.param, defaults)
        # self.param = list(param)
        # self.alpha = alpha
        # self.beta1 = beta1
        # self.beta2 = beta2
        # self.eps = eps

    def step(self):
        self.t += 1
        i = 0
        for group in self.param_groups:
          alpha, eps = group['lr'], group['eps']
          beta1, beta2 = group['betas']
          for par in group['params']:
            gt = par.grad.data
            self.m[i] = beta1 * self.m[i] + (1 - beta1) * gt
            self.v[i] = beta2 * self.v[i] + (1 - beta2) * (gt ** 2)
            mt = self.m[i] / (1 - beta1 ** self.t)
            vt = self.v[i] / (1 - beta2 ** self.t)
            par.data = par.data - alpha * mt / (vt ** 0.5 + eps)
            i += 1

In [102]:
#инициализация модели свёрточной нейронной сети, выбор функции потерь и оптимизитора
model = NeuralNetworks()
loss_func = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr = learn_rate, momentum = momentum)
optimizer = Adam(model.parameters(), alpha = learn_rate)
# for param in model.parameters():
#     print(param.numel(), param.size(), param.data.numpy())


In [103]:
#цикл обучения и проверки результатов обучения нейронной сети
for ep in range(epoch):
    print(f"Epoch {ep + 1}\n-------------------------------")
    train_loop(dataload_train, model, loss_func, optimizer)
    test_loop(dataload_test, model, loss_func)

Epoch 1
-------------------------------
loss: 1.902140  [ 2000/50000]
loss: 1.626977  [ 4000/50000]
loss: 1.514962  [ 6000/50000]
loss: 1.474470  [ 8000/50000]
loss: 1.438166  [10000/50000]
loss: 1.401290  [12000/50000]
Accuracy of the network on the 10000 test images: 50 %, loss: 1.3679435
Epoch 2
-------------------------------
loss: 1.340068  [ 2000/50000]
loss: 1.330802  [ 4000/50000]
loss: 1.336162  [ 6000/50000]
loss: 1.310423  [ 8000/50000]
loss: 1.308589  [10000/50000]
loss: 1.292068  [12000/50000]
Accuracy of the network on the 10000 test images: 53 %, loss: 1.3030076
Epoch 3
-------------------------------
loss: 1.224545  [ 2000/50000]
loss: 1.222666  [ 4000/50000]
loss: 1.233755  [ 6000/50000]
loss: 1.213878  [ 8000/50000]
loss: 1.209254  [10000/50000]
loss: 1.223192  [12000/50000]
Accuracy of the network on the 10000 test images: 56 %, loss: 1.2181051
Epoch 4
-------------------------------
loss: 1.148077  [ 2000/50000]
loss: 1.146092  [ 4000/50000]
loss: 1.165533  [ 6000/5

In [4]:
#сохранение модели
torch.save(model, 'model.pth')

NameError: name 'model' is not defined